
# AAMR Forecasting (ARIMA/SARIMAX) — Step‑by‑Step Notebook

This notebook forecasts **Age-Adjusted Mortality Rates (AAMR)** out to a target year using **statsmodels SARIMAX**.
It includes:
- Robust year parsing (handles dates and messy strings)
- Annual PeriodIndex (no "unsupported index" warnings)
- Option to handle 2020–2021 as **none / dummy / drop**
- Simple **auto-order** search using AICc
- Rolling backtest (**MAE/RMSE/MAPE**)
- Exports tidy CSVs, plots, and a manifest

> **How to use**: Edit the **Configuration** cell below (paths + options), then **Run All**.


## 0. (Optional) Install/Update Libraries
Run this if your environment is missing any dependencies.

In [ ]:

# If needed, uncomment and run:
# %pip install --upgrade pip
# %pip install pandas matplotlib statsmodels openpyxl xlrd


## 1. Imports & Settings

In [1]:

import os, re, glob, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox

warnings.filterwarnings("ignore", category=FutureWarning)
# Use non-interactive backend when running in batch; VS Code/Jupyter will still show plots inline
plt.rcParams['figure.figsize'] = (10, 5)


## 2. Configuration

In [2]:

# === EDIT THESE ===
DATA_DIR = r"E:\Collaboration Work\With Farooq\medical\data"  # <- folder with your Excel files
FILE_GLOB = "*.xlsx"            # e.g., "*.xlsx" or "AAMR_*.xlsx"
OUTPUT_DIR = "forecast_output"  # outputs go here (created if missing)

END_YEAR = 2035                 # forecast through this year
MIN_TRAIN = 12                  # minimum training length in years
PANDEMIC_MODE = "dummy"         # "none" | "dummy" | "drop"

# If the auto-target guess ever fails, set one of these:
TARGET_COL_OVERRIDE = None      # e.g., "Age Adjusted Rate"
TARGET_REGEX_OVERRIDE = None    # e.g., r"(?i)age\\s*adj.*rate"


## 3. Helper Functions — Parsing, Cleaning, Indexing

In [3]:

AAMR_GUESS_PATTERNS = [
    r"\bAAMR\b",
    r"age[-\s_]*adj(?:usted)?(?:[-\s_]*mortality)?[-\s_]*rate",
    r"age[-\s_]*adjusted.*rate",
    r"\bage[-\s_]*adj\b",
    r"\bage[-\s_]*standardi[sz]ed.*rate",
]
YEAR_ALIAS = [r"^year$", r"^yr$", r"^calendar\s*year$"]

def normalize_col(c): 
    import re
    return re.sub(r"\s+", " ", str(c)).strip()

def find_year_col(cols):
    import re
    low = [str(c).lower().strip() for c in cols]
    for pat in YEAR_ALIAS:
        for i,c in enumerate(low):
            if re.search(pat, c): 
                return cols[i]
    for i,c in enumerate(low):
        if "year" in c: 
            return cols[i]
    return None

def parse_year_series(s, lo=1900, hi=2100):
    """Extract 4-digit year from messy strings/dates, coerce invalid to NaN, bound to [lo,hi]."""
    if pd.api.types.is_datetime64_any_dtype(s):
        years = s.dt.year.astype("Float64")
    else:
        ss = s.astype(str)
        years = ss.str.extract(r"(\d{4})", expand=False)
        years = pd.to_numeric(years, errors="coerce")
    years = years.astype("Float64")
    years = years.where((years >= lo) & (years <= hi))
    return years

def find_target_col(df, explicit_name=None, target_regex=None):
    import re
    cols = list(df.columns)
    if explicit_name and explicit_name in cols:
        return explicit_name
    if target_regex:
        for c in cols:
            if re.search(target_regex, str(c), flags=re.I):
                return c
    scores=[]
    for c in cols:
        s=0; name=str(c)
        for pat in AAMR_GUESS_PATTERNS:
            if re.search(pat, name, flags=re.I): s+=2
        if re.search(r"\brate\b", name, flags=re.I): s+=1
        if re.search(r"crude", name, flags=re.I): s-=3
        if not pd.api.types.is_numeric_dtype(df[c]): s-=10
        scores.append((s,c))
    scores.sort(reverse=True)
    best = scores[0][1] if scores else None
    if best is not None and pd.api.types.is_numeric_dtype(df[best]): 
        return best
    return None

def clean_numeric(s):
    if pd.api.types.is_numeric_dtype(s): 
        out = pd.to_numeric(s, errors="coerce")
    else:
        out = (s.astype(str)
                 .str.replace(",","",regex=False)
                 .str.replace("%","",regex=False)
                 .str.extract(r"([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)")[0])
        out = pd.to_numeric(out, errors="coerce")
    return out.astype("Float64")

def ensure_dir(p): 
    Path(p).mkdir(parents=True, exist_ok=True)

def sanitize_filename(s): 
    import re
    return re.sub(r"[\\/:*?\"<>|]+","_",s)[:180]

def as_annual_series(y_values, years):
    idx = pd.PeriodIndex(np.asarray(years, dtype=int), freq='A-DEC')
    return pd.Series(np.asarray(y_values, dtype=float), index=idx)

def align_exog_to_years(exog_df, years):
    if exog_df is None:
        return None
    idx = pd.PeriodIndex(np.asarray(years, dtype=int), freq='A-DEC')
    X = exog_df.copy()
    X.index = pd.PeriodIndex(np.asarray(X.index, dtype=int), freq='A-DEC')
    return X.reindex(idx)

def make_intervention_exog(years, mode):
    if mode != "dummy": 
        return None
    y = np.array(years, dtype=int)
    X = pd.DataFrame({
        "d2020": (y == 2020).astype(int),
        "d2021": (y == 2021).astype(int)
    }, index=years)
    return X


## 4. Modeling Helpers — Stationarity, AICc Grid, Forecast

In [4]:

from statsmodels.tsa.statespace.sarimax import SARIMAX

def kpss_adf_diff_order(y, max_d=2):
    d=0; yv=pd.Series(y)
    while d<max_d and len(yv)>=3:
        adf_p = adfuller(yv, autolag="AIC")[1] if len(yv)>=5 else 1.0
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                kpss_p = kpss(yv, nlags="auto")[1] if len(yv)>=8 else 0.0
        except Exception:
            kpss_p = 0.0
        if adf_p<0.05 and kpss_p>0.05: break
        yv = yv.diff().dropna(); d+=1
    return d

def aicc(llf, k, n):
    return -2*llf + 2*k*n/(n-k-1) if (n-k-1)>0 else np.inf

def fit_sarimax_aicc(y_series, exog, d, p_range=range(0,6), q_range=range(0,6)):
    best=None; n=len(y_series)
    for p in p_range:
        for q in q_range:
            if p==0 and q==0 and d==0: 
                continue
            try:
                m=SARIMAX(y_series, order=(p,d,q), exog=exog,
                          trend="c", enforce_stationarity=False,
                          enforce_invertibility=False)
                res=None
                for method in ("lbfgs", "powell", "nm"):
                    try:
                        res=m.fit(disp=False, maxiter=500, method=method, concentrate_scale=True)
                        break
                    except Exception:
                        res=None
                if res is None: 
                    continue
                k=res.params.size; score=aicc(res.llf, k, n)
                if (best is None) or (score<best["aicc"]):
                    best={"res":res,"aicc":score,"order":(p,d,q)}
            except Exception:
                continue
    return best["res"] if best else None

def ljungbox_pvalue(residuals, lags=10):
    try:
        lb=acorr_ljungbox(residuals, lags=[lags], return_df=True)
        return float(lb["lb_pvalue"].iloc[0])
    except Exception:
        return np.nan

def predict_pi(res, n, exog_future=None, alpha=0.2):
    fc = res.get_forecast(steps=n, exog=exog_future)
    mean = fc.predicted_mean.values
    conf = fc.conf_int(alpha=alpha).values
    return mean, conf[:,0], conf[:,1]

def plot_history_forecast(series_name, years, y, years_fc, yhat, lo80, hi80, lo95, hi95, outpath):
    fig=plt.figure(figsize=(10,5))
    plt.plot(years, y, label="historical")
    if len(years_fc):
        plt.plot(years_fc, yhat, label="forecast")
        plt.fill_between(years_fc, lo80, hi80, alpha=0.3, label="80% PI")
        plt.fill_between(years_fc, lo95, hi95, alpha=0.2, label="95% PI")
    plt.title(series_name); plt.xlabel("Year"); plt.ylabel("AAMR"); plt.legend(); plt.tight_layout()
    fig.savefig(outpath, dpi=160); plt.close(fig)

def rolling_backtest(y, years, exog, min_train, h=1):
    preds=[]; actuals=[]; pred_years=[]
    for i in range(min_train, len(y)):
        y_train = as_annual_series(y[:i], years[:i])
        X_train = align_exog_to_years(exog.iloc[:i] if exog is not None else None, years[:i])
        X_test  = align_exog_to_years(exog.iloc[i:i+h] if exog is not None else None, years[i:i+h])
        d=kpss_adf_diff_order(y_train.values)
        res=fit_sarimax_aicc(y_train, X_train, d)
        if res is None: continue
        fc=res.get_forecast(steps=h, exog=X_test)
        preds.append(float(fc.predicted_mean[-1]))
        actuals.append(float(y[i])); pred_years.append(int(years[i]))
    preds=np.array(preds); actuals=np.array(actuals); pred_years=np.array(pred_years)
    mae=float(np.mean(np.abs(actuals-preds))) if len(preds) else np.nan
    rmse=float(np.sqrt(np.mean((actuals-preds)**2))) if len(preds) else np.nan
    mape=float(np.mean(np.abs((actuals-preds)/np.maximum(actuals,1e-12)))*100) if len(preds) else np.nan
    return pred_years, preds, actuals, {"MAE":mae,"RMSE":rmse,"MAPE":mape}


## 5. Core Pipeline — Read, Process, Forecast, Export

In [5]:

def process_one_dataframe(df0, meta_name, args, global_outputs):
    df=df0.copy(); df.columns=[normalize_col(c) for c in df.columns]

    # robust YEAR
    year_col=find_year_col(df.columns)
    if year_col is None:
        print(f"[skip] {meta_name}: no Year col."); return
    yrs = parse_year_series(df[year_col])
    df = df.assign(__YEAR__=yrs).dropna(subset=["__YEAR__"])
    df["__YEAR__"] = df["__YEAR__"].astype(int)

    # target
    target_col=find_target_col(df, explicit_name=args.get("target_col"), target_regex=args.get("target_regex"))
    if target_col is None:
        print(f"[skip] {meta_name}: no AAMR col."); return
    df[target_col]=clean_numeric(df[target_col])
    df=df.dropna(subset=[target_col])
    df=df[np.isfinite(df[target_col])].sort_values("__YEAR__")

    # training mask for pandemic
    if args["pandemic_mode"]=="drop":
        train_mask=~df["__YEAR__"].isin([2020,2021])
    else:
        train_mask=np.ones(len(df), dtype=bool)

    fit_and_forecast_series(df, "__YEAR__", target_col, meta_name, "ALL", train_mask, args, global_outputs)

def fit_and_forecast_series(sub, year_col, target_col, file_sheet_name, series_key, train_mask, args, global_outputs):
    out_dir=args["output_dir"]
    ensure_dir(out_dir)

    series_name=f"{file_sheet_name} :: {series_key}"
    series_id=sanitize_filename(series_name)

    y_all=sub[target_col].astype(float).values
    years_all=sub[year_col].astype(int).values

    idx=np.where(train_mask)[0]
    y,years=y_all[idx],years_all[idx]
    if len(y)<max(args["min_train"],6):
        print(f"[skip] {series_name}: too few points ({len(y)})"); return

    # backtest
    X_train0=make_intervention_exog(years, args["pandemic_mode"])
    bt_years,_,_,bt_metrics=rolling_backtest(y, years, X_train0, min_train=args["min_train"], h=1)

    # final fit on training window
    y_train_series = as_annual_series(y, years)
    X_train = align_exog_to_years(X_train0, years)
    d=kpss_adf_diff_order(y_train_series.values)
    res=fit_sarimax_aicc(y_train_series, X_train, d)
    if res is None:
        print(f"[warn] {series_name}: model fail"); return

    # forecast horizon
    last_year=int(years_all.max())
    horizon=max(0, int(args["end_year"])-last_year)
    years_fc=np.arange(last_year+1, last_year+1+horizon, dtype=int)

    if horizon>0:
        X_future=make_intervention_exog(years_fc, args["pandemic_mode"])
        X_future=align_exog_to_years(X_future, years_fc)
        yhat80, lo80, hi80 = predict_pi(res, horizon, X_future, alpha=0.20)
        yhat95, lo95, hi95 = predict_pi(res, horizon, X_future, alpha=0.05)
    else:
        years_fc=[]; yhat95=lo80=hi80=lo95=hi95=[]

    # export rows
    out_rows=[]
    for yr,val in zip(years_all,y_all):
        out_rows.append({"series":series_name,"year":int(yr),"aamr":float(val),
                         "is_forecast":0,"lo80":np.nan,"hi80":np.nan,
                         "lo95":np.nan,"hi95":np.nan})
    for i in range(len(years_fc)):
        out_rows.append({"series":series_name,"year":int(years_fc[i]),
                         "aamr":float(yhat95[i]),"is_forecast":1,
                         "lo80":float(lo80[i]),"hi80":float(hi80[i]),
                         "lo95":float(lo95[i]),"hi95":float(hi95[i])})
    out_df=pd.DataFrame(out_rows)

    out_csv=os.path.join(out_dir,"series_csv",f"{series_id}.csv")
    ensure_dir(os.path.dirname(out_csv)); out_df.to_csv(out_csv,index=False)

    plot_path=os.path.join(out_dir,"plots",f"{series_id}.png")
    ensure_dir(os.path.dirname(plot_path))
    plot_history_forecast(series_name, years_all, y_all, years_fc, yhat95, lo80, hi80, lo95, hi95, plot_path)

    # metrics + index
    global_outputs["metrics"].append({"series":series_name,"file_sheet":file_sheet_name,
                                      "target_col":target_col,"n_obs":int(len(y_all)),
                                      "train_n":int(len(y)),
                                      "last_year":int(years_all.max()), **bt_metrics})
    global_outputs["index"].append({"series":series_name,
                                    "csv":os.path.relpath(out_csv, out_dir).replace("\","/"),
                                    "plot":os.path.relpath(plot_path, out_dir).replace("\","/")})

def load_excel_as_sheets(path):
    sheets=pd.read_excel(path, sheet_name=None, engine="openpyxl")
    return {f"{os.path.basename(path)} :: {s}":df for s,df in sheets.items()}


SyntaxError: unterminated string literal (detected at line 94) (2666599403.py, line 94)

## 6. Run the Pipeline

In [6]:

# Build args dict
args = dict(
    data_dir=DATA_DIR,
    file_glob=FILE_GLOB,
    output_dir=OUTPUT_DIR,
    end_year=END_YEAR,
    min_train=MIN_TRAIN,
    pandemic_mode=PANDEMIC_MODE,
    target_col=TARGET_COL_OVERRIDE,
    target_regex=TARGET_REGEX_OVERRIDE,
)

# Gather files
files = sorted(glob.glob(os.path.join(args["data_dir"], args["file_glob"])))
if not files:
    raise SystemExit("No files found. Check DATA_DIR and FILE_GLOB.")

global_outputs={"metrics":[], "index":[]}

for fpath in files:
    try:
        sheets=load_excel_as_sheets(fpath)
    except Exception as e:
        print(f"[skip] {os.path.basename(fpath)}: cannot read ({e})")
        continue

    for tag, df in sheets.items():
        try:
            process_one_dataframe(df, tag, args, global_outputs)
        except Exception as e:
            print(f"[warn] processing failed for {tag}: {e}")

# Write outputs
ensure_dir(args["output_dir"])
metrics_path = os.path.join(args["output_dir"], "metrics.csv")
pd.DataFrame(global_outputs["metrics"]).to_csv(metrics_path, index=False)

manifest_path = os.path.join(args["output_dir"], "manifest.json")
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(global_outputs, f, indent=2)

print("Done.")
print(f"- Metrics: {metrics_path}")
print(f"- Series CSVs: {os.path.join(args['output_dir'],'series_csv')}")
print(f"- Plots: {os.path.join(args['output_dir'],'plots')}")
print(f"- Manifest: {manifest_path}")


[skip] Obesity+DM+HTN POD corrected.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] Obesity+DM+HTN age groups final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] Obesity+DM+HTN states corrected.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] Obesity+DM+HTN urbanization final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] obesity+DM+HTN census region final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] obesity+DM+HTN gender final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] obesity+DM+HTN overall final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] obesity+DM+HTN race final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
[skip] ~$Obesity+DM+HTN age groups final.xlsx: cannot read (name 'load_excel_as_sheets' is not defined)
Done.
- Metrics: forecast_output\metrics.csv
- Series CSVs: forecast_output\series_csv
- Pl

## 7. Quick Checks — Metrics & First Plot

In [7]:

from IPython.display import display, Image

# Show metrics head
m = pd.read_csv(os.path.join(OUTPUT_DIR, "metrics.csv"))
display(m.head(10))

# Try to display the first available plot
try:
    import os
    plots_dir = os.path.join(OUTPUT_DIR, "plots")
    plot_files = [os.path.join(plots_dir, x) for x in os.listdir(plots_dir) if x.endswith(".png")]
    if plot_files:
        display(Image(filename=plot_files[0]))
except Exception as e:
    print("Could not display plot:", e)


EmptyDataError: No columns to parse from file


## 8. Troubleshooting

- **No files found**: Check `DATA_DIR` and `FILE_GLOB` in the config cell.
- **Target column not found**: Set `TARGET_COL_OVERRIDE` to the exact AAMR column name, or `TARGET_REGEX_OVERRIDE` (e.g., `r"(?i)age\s*adj.*rate"`).
- **Convergence warnings**: Normal; the notebook tries multiple optimizers. If persistent, increase training years or try `PANDEMIC_MODE="drop"`.
- **Weird spikes around 2020–2021**: Try `PANDEMIC_MODE="dummy"` (default) or `PANDEMIC_MODE="drop"`.
- **Accuracy**: Open `metrics.csv` and look at `MAPE/MAE/RMSE`. We can add ETS fallback if any series still fails.
